<a href="https://colab.research.google.com/github/daniel-falk/ai-ml-principles-exercises/blob/main/ML-training/intro-to-libraries/benchmarking_and_numba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Benchmarking python code
Benchmarking should not only be done when optimizing scripts but also at each change of bottleneck code to monitor the efficiency over time.

Lets start with creating a large array.

In [ ]:
import random

large_array = [random.random() for i in range(10_000_000)]

## Benchmarking using the naive method
The library `time` can be used to get the current time. We can use that to measure the difference in time before and after an operation.

In [ ]:
import time

t0 = time.time()
new_array = [v / 2 for v in large_array]
t1 = time.time()

print(f"Operation took {(t1 - t0) * 1_000 : .2f} mS")

Run the code multiple times and you will se that the duration differs due to e.g. current load of the computer.

To deal with this we can run the experiment multiple times.

In [ ]:
durations = []
for _ in range(10):
  t0 = time.time()
  new_array = [v / 2 for v in large_array]
  t1 = time.time()
  durations.append((t1 - t0) * 1_000)

print(f"Operation took in average {sum(durations) / len(durations): .2f} mS from {len(durations)} experiments")

What if the duration of the operation changes significantly? Then we need to change the number of experiments we run and also the convertion of the units we present the number in.

In [ ]:
import itertools

# Concatenate the large array ten time to create a 10x larger array
very_large_array = list(itertools.chain(*[large_array for _ in range(10)]))

durations = []
for _ in range(3):
  t0 = time.time()
  new_array = [v / 2 for v in very_large_array]
  t1 = time.time()
  durations.append(t1 - t0)

print(f"Operation took in average {sum(durations) / len(durations): .2f} S from {len(durations)} experiments")

# Benchmarking with 'timeit'
We do not have to invent all this benchmarking code over and over again in every project we do. There is a built-in module called `timeit` that can help us with it.

In [ ]:
import timeit

def my_function():
  return [v / 2 for v in large_array]

experiments = 10
mean_duration = timeit.timeit(my_function, number=experiments)
print(f"Operation took in average {mean_duration: .2f} S from {experiments} experiments")

We got rid of some code, but the two issues stated above still stands. If we are using notebooks, e.g. `Colab` or `Jupyter`, then we can use the magic function `%time` to measure the execution time of a single line, or `%%time` to measure the execution time of the full cell.

In [ ]:
%time new_array = [v / 2 for v in large_array]

In [ ]:
%%time

new_array = [v / 2 for v in large_array]
print(len(new_array))

We can also use the magic functions `%timeit` and `%%timeit` that will measaure multiple experiments to get the mean time (and the variance).

In [ ]:
%timeit new_array = [v / 2 for v in large_array]

In [ ]:
%%timeit

new_array = [v / 2 for v in large_array]
print(len(new_array))

The `%timeit` magic functions will automatically select a number of iterations to run based on how much time each experiment takes.

In [ ]:
%timeit new_array = [v / 2 for v in large_array[:10]]

You can manually override the number of runs and loops in each run with the `-r` and `-n` options.

In [ ]:
%timeit -r 2 -n 2 new_array = [v / 2 for v in large_array]

# Optimizing code with numba
We will later see how much code, especially vector operations, can be optimized using libraries such as `numpy`, but some code will not be helped by `numpy`. In these cases, just in time compilation with e.g. `numba` can be used.

In [ ]:
import numba

In [ ]:
def my_function(max_val):
  sum_of_even = 0
  for value in range(max_val):
    if value % 2 == 0:
      sum_of_even += value
  return sum_of_even

%timeit my_function(1_000_000)

In [ ]:
optimized = numba.njit(my_function)
%timeit optimized(1_000_000)

Be aware that the compilation takes place the first time the function is called. For some tasks the compilation step might take more time than the full execution of the function calls, but if called many times it is probably worth it, in that case you might want to call the function once before benchmarking it.

In [ ]:
optimized = numba.njit(my_function)
%time optimized(1_000_000)
%timeit optimized(1_000_000)